In [1]:
import os
import random
import json
import csv
import numpy as np
import pandas as pd
from collections import namedtuple
from tqdm.auto import tqdm
from pathlib import Path
from datetime import datetime

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertForTokenClassification, BertTokenizer, AutoModelWithLMHead, AutoConfig
import pytorch_lightning as pl

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

import warnings
warnings.filterwarnings('ignore')

import pdb

## Download pretrained model and tokenizer

From https://huggingface.co/models

In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer

In [8]:
# Download model/tokenizer, or load from cache
config = AutoConfig.from_pretrained(
    'bert-large-uncased',
    cache_dir='./cache',
    output_past = False,
    num_labels=3,  # change based on number of IOB labels
)
model = BartForConditionalGeneration.from_pretrained(
    'bert-large-uncased',
    config=config,
    cache_dir='./cache'
)
tokenizer = BartTokenizer.from_pretrained(
    'bert-large-uncased',
    cache_dir='./cache'
)

In [2]:
MODEL_DIR = "./bert-large-uncased-25Mar2020"

In [10]:
# Save the model/tokenizer
model.save_pretrained(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)
config.save_pretrained(MODEL_DIR)

('./bart-large-25Mar2020/vocab.json',
 './bart-large-25Mar2020/merges.txt',
 './bart-large-25Mar2020/special_tokens_map.json',
 './bart-large-25Mar2020/added_tokens.json')

In [3]:
# Load the model/tokenizer
config = AutoConfig.from_pretrained(
    MODEL_DIR, 
    cache_dir='./cache',
    output_past = False,
)
tokenizer = BartTokenizer.from_pretrained(
    MODEL_DIR, 
    cache_dir='./cache'
)
model = BartForConditionalGeneration.from_pretrained(
    MODEL_DIR, 
    config=config,
    cache_dir='./cache'
)

KeyboardInterrupt: 

## Load BERT model for training

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at ./cache/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
INFO:transformers.configuration_utils:Model config BertConfig {
  "_num_labels": 3,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_len

In [5]:
model = BertForTokenClassification.from_pretrained(
    'bert-large-uncased',
    from_tf=False,
    config=config,
    cache_dir='./cache',
)

INFO:filelock:Lock 47013707973240 acquired on ./cache/54da47087cc86ce75324e4dc9bbb5f66c6e83a7c6bd23baea8b489acc8d09aa4.4d5343a4b979c4beeaadef17a0453d1bb183dd9b084f58b84c7cc781df343ae6.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-pytorch_model.bin not found in cache or force_download set to True, downloading to /hps/nobackup/research/chembl/deeplearning/transformers-ner/cache/tmpvu7vxfdd
Downloading: 100%|██████████| 1.34G/1.34G [08:59<00:00, 2.49MB/s] 
INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-pytorch_model.bin in cache at ./cache/54da47087cc86ce75324e4dc9bbb5f66c6e83a7c6bd23baea8b489acc8d09aa4.4d5343a4b979c4beeaadef17a0453d1bb183dd9b084f58b84c7cc781df343ae6
INFO:transformers.file_utils:creating metadata file for ./cache/54da47087cc86ce75324e4dc9bbb5f66c6e83a7c6bd23baea8b489acc8d09aa4.4d5343a4b979c4beeaadef17a0453d1bb183dd9b084f58b84c7cc781df343ae6
INFO:filelock

In [48]:
import itertools as it

def get_token_text_locs(text, tokens):
    tok_i = 0
    tok_char_i = 0
    char_toks = []
    for i,c in enumerate(text.lower()):            
        if tok_char_i > len(tokens[tok_i])-1:
            tok_i += 1
            tok_char_i = 0
            
#         For ROBERTA
#         while True:
#             if tok_i > len(tokens)-1:
#                 break
#             if tokens[tok_i] in {'Ċ', 'Ġ'}:
#                 tok_i += 1
#                 tok_char_i = 0
#             else:
#                 break

        if tok_i > len(tokens)-1:
            break

        if tokens[tok_i][0:2] == '##' and tok_char_i == 0:
            tok_char_i = 2
            
#         For ROBERTA
#         if tokens[tok_i][0] == 'Ġ' and tok_char_i == 0:
#             tok_char_i = 1

        if tokens[tok_i][tok_char_i] == c:
            tok_char_i += 1

        char_toks.append(tok_i)

    return char_toks

def remove_overlaps(locs):
    remove = set()
    for (b1,e1),(b2,e2) in it.permutations(locs,2):
        if b2>=b1 and e2<=e1:
            if (e2-b2)<(e1-b1):
                remove.add((b2,e2))

    return [(b,e) for b,e in locs if not (b,e) in remove]

def get_text_locs(s, text):
    s = s.lower()
    text = text.lower()
    l = len(s)
    matches = []
    for i in range(len(text)):
        if text[i:i+l] == s:
            matches.append((i,i+l))
    return matches

def tokenize(tokenizer, text):
    tokens = tokenizer.tokenize(text.lower(), add_special_tokens=True)
    codes = tokenizer.encode(text.lower(), add_special_tokens=True)

    return tokens, codes

def gen_iob(tokenizer, text, targets):
    tokens, codes = tokenize(tokenizer, text)
    char_toks = get_token_text_locs(text, tokens)
    
    matches = []
    for s in targets:
        matches.append(get_text_locs(s,text))
    matches = sorted([l for m in matches for l in m])
    matches = remove_overlaps(matches)

    iob = [[t,"O"] for t in tokens]
    match_toks = [i for b,e in matches for i in range(char_toks[b],char_toks[e-1]+1)]
    for l in match_toks:
        iob[l][1] = "I-IND"
    match_beginnings = [char_toks[b] for b,e in matches]
    for l in match_beginnings:
        iob[l][1] = "B-IND"
    
    for i,(t,l) in enumerate(iob):
        if t[:2] == '##':
            l = '[PAD]'
        iob[i] = (t,l)

    return iob, codes

In [49]:
s = "hello, this is (a) testing, <bullet> et cetera. This is another sentence. <newline> OK then"

In [51]:
tokens, codes = tokenize(tokenizer, s)

In [52]:
tokens

['hello',
 ',',
 'this',
 'is',
 '(',
 'a',
 ')',
 'testing',
 ',',
 '<bullet>',
 'et',
 'ce',
 '##tera',
 '.',
 'this',
 'is',
 'another',
 'sentence',
 '.',
 '<newline>',
 'ok',
 'then']

In [53]:
gen_iob(tokenizer, s, ['this is', 'et cetera'])

([('hello', 'O'),
  (',', 'O'),
  ('this', 'B-IND'),
  ('is', 'I-IND'),
  ('(', 'O'),
  ('a', 'O'),
  (')', 'O'),
  ('testing', 'O'),
  (',', 'O'),
  ('<bullet>', 'O'),
  ('et', 'B-IND'),
  ('ce', 'I-IND'),
  ('##tera', '[PAD]'),
  ('.', 'O'),
  ('this', 'B-IND'),
  ('is', 'I-IND'),
  ('another', 'O'),
  ('sentence', 'O'),
  ('.', 'O'),
  ('<newline>', 'O'),
  ('ok', 'O'),
  ('then', 'O')],
 [101,
  7592,
  1010,
  2023,
  2003,
  1006,
  1037,
  1007,
  5604,
  1010,
  30524,
  3802,
  8292,
  14621,
  1012,
  2023,
  2003,
  2178,
  6251,
  1012,
  30523,
  7929,
  2059,
  102])

In [1]:
from utils import *
import yaml

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

args_file = "./args.yaml"

with open(args_file) as f:
    args_dict = yaml.load(f)

extra_args_file = ""
    
if extra_args_file:
    with open(extra_args_file) as f:
        extra_args_dict = yaml.load(f)
        for k,v in extra_args_dict.items():
            args_dict[k] = v

args = Dict2Obj(args_dict)

checkpoint_dir = "../bert-lm/output/14May2020-17h05m15s/checkpoint-1584"
args['model_name_or_path'] = checkpoint_dir
args['config_name'] = checkpoint_dir
args['tokenizer_name'] = checkpoint_dir

bert_system = BertNerSystem(args)  # Load model
trainer = get_trainer(bert_system, args)  # get the trainer

## Test dataloader

In [3]:
import pdb, traceback, sys
try:
    dataloader = bert_system.train_dataloader()
except:
    extype, value, tb = sys.exc_info()
    traceback.print_exc()
    pdb.post_mortem(tb)

Loading train data: 100%|██████████| 1088/1088 [00:08<00:00, 130.74it/s]


In [4]:
dataset = dataloader.dataset

In [7]:
next(iter(dataloader))

{'tokens': tensor([[  101, 24936,  1998,  8192,  1026,  2047,  4179,  1028,  3161, 10768,
           2003,  5393,  2005,  1996,  9740,  1998,  3949,  1997,  3707, 18888,
           2019, 17577,  1998,  1013,  2030, 28268, 13164,  4135, 28522, 10074,
           2019, 17577,  2015,  1012,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,

In [5]:
len(dataset.data)

903

In [6]:
dataset.data[0]

['4f58c203-f649-a18e-aaf3-70b0797652f4',
 '1 INDICATIONS AND USAGE<newline>  Azelastine hydrochloride nasal spray is indicated for the treatment of the symptoms of seasonal allergic rhinitis in adults and pediatric patients 5 years and older, and for the treatment of the symptoms of vasomotor rhinitis in adults and adolescent patients 12 years and older.<newline>  Azelastine hydrochloride nasal spray is an H1-receptor antagonist indicated for the treatment of the symptoms of seasonal allergic rhinitis in adults and pediatric patients 5 years and older and for the treatment of the symptoms of vasomotor rhinitis in adults and adolescent patients 12 years and older. ( 1 ) ',
 ['SEASONAL ALLERGIC RHINITIS', 'VASOMOTOR RHINITIS'],
 [('1', 'O'),
  ('indications', 'O'),
  ('and', 'O'),
  ('usage', 'O'),
  ('<', 'O'),
  ('new', 'O'),
  ('##line', 'O'),
  ('>', 'O'),
  ('az', 'O'),
  ('##ela', 'O'),
  ('##sti', 'O'),
  ('##ne', 'O'),
  ('hydro', 'O'),
  ('##ch', 'O'),
  ('##lor', 'O'),
  ('##id

## Test training

In [2]:
import pdb, traceback, sys
try:
    trainer.fit(bert_system)
except:
    extype, value, tb = sys.exc_info()
    traceback.print_exc()
    pdb.post_mortem(tb)

Loading val data: 100%|██████████| 543/543 [00:04<00:00, 132.99it/s]

Validation sanity check: 0it [00:00, ?it/s]

Loading train data:   0%|          | 0/1088 [00:00<?, ?it/s]

Loading val data: 100%|██████████| 543/543 [00:03<00:00, 136.71it/s]

Epoch 1:   0%|          | 0/919 [00:00<?, ?it/s] 

...

Epoch 16:  94%|█████████▍| 862/919 [02:20<00:09,  6.14it/s, loss=0.000, v_num=33]

INFO:utils:***** Validation results *****
INFO:utils:avg_val_loss = tensor(0.1794, device='cuda:0')

INFO:utils:epoch = 15

INFO:utils:loss = tensor(0.0004, device='cuda:0')



Epoch 16: 100%|██████████| 919/919 [02:34<00:00,  5.95it/s, loss=0.000, v_num=33]

INFO:utils:Model checkpointed at step 288


Epoch 16: 100%|██████████| 919/919 [02:40<00:00,  5.73it/s, loss=0.000, v_num=33]
